# Imports

In [1]:
from datasets import load_dataset
from collections import Counter
import torch
import torch.nn as nn
from utils.datautils import *
from utils.MLutils import *
from utils.resources import *
from transformers import BertTokenizerFast
from sklearn.model_selection import train_test_split
from transformers import BertModel
import unicodedata

# linea que arregla algunos errores de loadeo de datasets
#pip install --upgrade datasets

# linea que corre torch acelerado en mac
# pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

/home/fabro/Documents/uba/aprendizaje_automatico/Puntualizacion-Capitalizacion/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Procesamiento

In [2]:
linux = True
device = None

if linux:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
else:
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print("usando:", device)

usando: cuda


In [3]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

# Ejericio b)

## Busqueda de fuentes

- Fuente 1: Conjunto de preguntas en espa;ol
- Fuente 2: Dataset provisto para Notebook 10
- Fuente 3: Dataset sintetico generado con Gemini
- Fuente 4: Articulos de Wikipedia
- Fuente 5: Subtitulos de peliculas
- Fuente 6: Mixture of preguntas y afirmaciones

In [7]:
questions, question_for_mixture = get_questions()
oraciones_rnn = get_notebook_dataset()
oraciones_sinteticas = get_gemini_dataset()
frases_wikipedia = get_wikipedia_dataset()
esperando_la_carroza, frases_relatos_salvajes = get_pelis_dataset()

cant_oraciones = len(oraciones_sinteticas)
question_for_mixture = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", q).strip()) for q in question_for_mixture]
oraciones_sinteticas = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", a).strip()) for a in oraciones_sinteticas]

tanda_1 = question_for_mixture[:cant_oraciones]
question_affirmation = [f"{q} {a}" for q, a in zip(tanda_1, oraciones_sinteticas)]

tanda_2 = question_for_mixture[cant_oraciones:2*cant_oraciones]
affirmation_question = [f"{a} {q}" for q, a in zip(tanda_2, oraciones_sinteticas)]

tanda_3 = question_for_mixture[2*cant_oraciones:3*cant_oraciones]
tanda_3_shuffled = random.sample(tanda_3, len(tanda_3))
question_question = [f"{q} {p}" for q, p in zip(tanda_3, tanda_3_shuffled)]

mixtures = question_affirmation + affirmation_question + question_question

random.sample(mixtures, 5)

Se descargaron 5000 preguntas en Español.
Se descargaron 997 oraciones en Español (del dataset del notebook 10).
Hay 1413 oraciones sintéticas.
['Argentina, oficialmente República Argentina,[a]\u200b es un país soberano de América del Sur, ubicado en el extremo sur y sudeste de ese subcontinente.', 'Adopta la forma de gobierno republicana, democrática, representativa y federal.', 'Poseen Carta Magna, bandera y fuerzas de seguridad propias, el dominio de los recursos naturales circunscriptos en su territorio y delegan los poderes exclusivos al Gobierno Federal.', 'Hasta mediados del siglo XX, fue una de las economías más prósperas del mundo.', 'No obstante, es la segunda economía más importante de Sudamérica —detrás de Brasil— y la 24.º más grande del mundo por PIB nominal.']
✅ Se extrajeron 947 frases completas y se guardaron en 'dialogos_esperando_la_carroza.json'
✅ Frases extraídas y guardadas. Total: 947
['La canilla está cerrada.', '¿Qué te dijo?', 'Ah... Ahora se acuerdan de su bu

['Mi amigo de Nicaragua, llamado Javier, es un poeta muy reconocido. ¿Qué ha solicitado Cándido Méndez?',
 '¿Quiénes eran los personajes que podían verse habitualmente en las obras shunga? Quiero una doble Whopper de Burger King.',
 '¿Cuál creen los historiadores actuales que es el motivo del carácter de Calígula? ¿Quién es Ana Botella?',
 'El logo de Adidas Originals es un trébol. ¿Qué organización humanitaria tuvo origen en Suiza?',
 '¿Cómo es el campus de la Ciudadela Robledo? ¿Dónde se presentó El alquimista impaciente?']

## Juntamos las fuentes

In [8]:
oraciones_raw = questions + oraciones_rnn + oraciones_sinteticas + frases_wikipedia + esperando_la_carroza  + frases_relatos_salvajes + mixtures

print('Cantidad total de oraciones:',len(oraciones_raw))
print('Cantidad de oraciones de preguntas:',len(questions))
print('Cantidad de oraciones en espa;ol de hugging face:',len(oraciones_rnn))
print('Cantidad de oraciones sintéticas:',len(oraciones_sinteticas))
print('Cantidad de oraciones de Wikipedia:',len(frases_wikipedia))
print('Cantidad de oraciones de Esperando la carroza:',len(esperando_la_carroza))
print('Cantidad de oraciones de Relatos Salvajes:',len(frases_relatos_salvajes))
print('Cantidad de oraciones de mixture:',len(mixtures))

print("Algunas oraciones aleatorias:")
random.sample(oraciones_raw, 5)

Cantidad total de oraciones: 20244
Cantidad de oraciones de preguntas: 5000
Cantidad de oraciones en espa;ol de hugging face: 997
Cantidad de oraciones sintéticas: 1413
Cantidad de oraciones de Wikipedia: 6648
Cantidad de oraciones de Esperando la carroza: 947
Cantidad de oraciones de Relatos Salvajes: 1000
Cantidad de oraciones de mixture: 4239
Algunas oraciones aleatorias:


['El nuevo año comenzó con problemas, debido a que la comisión directiva del club había decidido contratar a Carlos Bilardo como director técnico.',
 '¿Que hace? Pera',
 'F., mientras que Messi hacía lo propio con el F.',
 'Ai, la concha de la novia.',
 'Pero aún así, el día de la mujer debe celebrarse como lo son otras fiestas".[45]\u200b\nEn 1975, la ONU celebró el Año Internacional de la Mujer.']

Separamos en conjuntos de `train` y `test` con el tokenizer de `BERT`

In [9]:
train_sents, test_sents = train_test_split(oraciones_raw, test_size=0.05, random_state=42)

dataloader_train = get_dataloader(oraciones_raw=oraciones_raw, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)
dataloader_test = get_dataloader(oraciones_raw=test_sents, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)

print(len(train_sents))
print(len(test_sents))

19231
1013


## Importamos el modelo

In [ ]:
from train.RNN import PunctuationCapitalizationRNN

model_name = "bert-base-multilingual-cased"
bert_model = BertModel.from_pretrained(model_name)

for param in bert_model.parameters():
    param.requires_grad = False

for param in bert_model.embeddings.word_embeddings.parameters():
    param.requires_grad = False

N = 2
for layer in bert_model.encoder.layer[-N:]:
    for param in layer.parameters():
        param.requires_grad = True

# ver que es pooler
for param in bert_model.pooler.parameters():
    param.requires_grad = True

### Entrenamiento

In [ ]:
model = PunctuationCapitalizationRNN(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_start_classes=len(PUNCT_START_TAGS),
    num_punct_end_classes=len(PUNCT_END_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)


punct_start_counter = Counter()
punct_end_counter = Counter()
cap_counter = Counter()
for input_ids, attention_mask, punct_start_labels, punct_end_labels, cap_labels in dataloader_train:
    punct_start_np = punct_start_labels.cpu().numpy()
    punct_end_np   = punct_end_labels.cpu().numpy()
    cap_np         = cap_labels.cpu().numpy()

    valid_start = punct_start_np[punct_start_np != -100]
    valid_end   = punct_end_np[punct_end_np != -100]
    valid_cap   = cap_np[cap_np != -100]

    punct_start_counter.update(valid_start)
    punct_end_counter.update(valid_end)
    cap_counter.update(valid_cap)

total_start = sum(punct_start_counter.values())
total_end   = sum(punct_end_counter.values())
total_cap   = sum(cap_counter.values())
beta = 0.7

start_weights = {
    tag: (total_start / count)**beta
    for tag, count in punct_start_counter.items()
}
end_weights = {
    tag: (total_end / count)**beta
    for tag, count in punct_end_counter.items()
}
cap_weights = {
    tag: (total_cap / count)**beta
    for tag, count in cap_counter.items()
}

start_weights_tensor = torch.tensor(
    [start_weights.get(i, 1.0) for i in range(len(PUNCT_START_TAGS))],
    dtype=torch.float32
).to(device).clamp(min=1.0, max=5.0)

end_weights_tensor = torch.tensor(
    [end_weights.get(i, 1.0) for i in range(len(PUNCT_END_TAGS))],
    dtype=torch.float32
).to(device).clamp(min=1.0, max=5.0)

cap_weights_tensor = torch.tensor(
    [cap_weights.get(i, 1.0) for i in range(len(CAP_TAGS))],
    dtype=torch.float32
).to(device).clamp(min=1.0, max=5.0)


criterion_start = nn.CrossEntropyLoss(ignore_index=-100, weight=start_weights_tensor)
criterion_end   = nn.CrossEntropyLoss(ignore_index=-100, weight=end_weights_tensor)
criterion_cap   = nn.CrossEntropyLoss(ignore_index=-100, weight=cap_weights_tensor)

trainable_params = [
    p for p in bert_model.parameters() if p.requires_grad
] + list(model.projection.parameters()) \
  + list(model.rnn.parameters()) \
  + list(model.punct_start_classifier.parameters()) \
  + list(model.punct_end_classifier.parameters()) \
  + list(model.cap_classifier.parameters())

optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)

train(model, dataloader_train=dataloader_train, dataloader_test=dataloader_test,optimizer=optimizer, criterion_start=criterion_start,criterion_end=criterion_end, criterion_cap = criterion_cap, device=device, epochs=20)

### Modelo cargado

In [14]:
model = PunctuationCapitalizationRNN(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_start_classes=len(PUNCT_START_TAGS),
    num_punct_end_classes=len(PUNCT_END_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)

model.load_state_dict(torch.load("model_rnn_tres_cabezas.pt", map_location=device))

<All keys matched successfully>

In [20]:
entrada = "no te lo puedo creer messi se va a retirar"

print(f"Entrada: {entrada}")
print(f"Salida: {predict_and_reconstruct(model, entrada, tokenizer, device, verbose=False)}")

Entrada: no te lo puedo creer messi se va a retirar
Salida: No, te lo puedo, creer, Messi, se va a retirar.


## Evaluacion

In [10]:
evaluate(model, dataloader_test, device)

Start Acc: 0.9980
End   Acc: 0.9709
Cap   Acc: 0.9840

Start report:
              precision    recall  f1-score   support

           Ø       1.00      1.00      1.00     21732
           ¿       0.95      0.98      0.96       579

    accuracy                           1.00     22311
   macro avg       0.97      0.99      0.98     22311
weighted avg       1.00      1.00      1.00     22311


End report:
              precision    recall  f1-score   support

           Ø       1.00      0.97      0.99     20589
           ,       0.49      0.90      0.64       500
           .       0.86      0.94      0.90       650
           ?       0.92      0.97      0.95       572

    accuracy                           0.97     22311
   macro avg       0.82      0.95      0.87     22311
weighted avg       0.98      0.97      0.97     22311


Cap report:
              precision    recall  f1-score   support

       lower       0.99      0.99      0.99     15241
        init       0.96      0.98 

In [11]:
entrada = "hola que lindo dia que hace no te parece"
print(f"{entrada} => {predict_and_reconstruct(model, entrada, tokenizer, device, verbose=False)}")

hola que lindo dia que hace no te parece => Hola, que lindo, Dia, que hace, no te parece?


## Prueba de control de Overfitting

explicacion de del corro

In [ ]:
frases = ["¿QUE?"]
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

train_loader = get_dataloader(frases, max_length=25, batch_size=1, device=device,tokenizer=tokenizer)

model = PunctuationCapitalizationRNN(
    bert_model=bert_model,
    hidden_dim=64,
    num_punct_start_classes=5,
    num_punct_end_classes=5,
    num_cap_classes=4
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)  # Alto LR

criterion_punct_start = nn.CrossEntropyLoss(ignore_index=-100)
criterion_punct_end   = nn.CrossEntropyLoss(ignore_index=-100)
criterion_cap         = nn.CrossEntropyLoss(ignore_index=-100)

train(
    model,
    dataloader_train=train_loader,
    dataloader_test=train_loader,
    optimizer=optimizer,
    criterion_start=criterion_punct_start,
    criterion_end=criterion_punct_end,
    criterion_cap=criterion_cap,
    device=device,
    epochs=200
)

entrada = "que"
print("Predicción:", predict_and_reconstruct(model, entrada, tokenizer, device))


## Export modelo

In [ ]:
torch.save(model.state_dict(), "model_rnn_tres_cabezas.pt")

## Generacion CSV TP

In [12]:
from utils.datautils import *

In [ ]:
# Ruta a un archivo TXT con párrafos (una instancia por párrafo)
ruta_txt = "predict/prueba.txt"

# Ejecutar predicciones y guardar CSV
df_predicciones = predicciones_TP(ruta_txt, model, tokenizer, device, max_length=128, verbose=True)

# Mostrar las primeras filas del dataframe con las predicciones
print(df_predicciones)